# Gathering Data

Importing requried librarys

In [5]:
%%writefile ../src/datautil.py
#datautil.py
import os
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

Overwriting ../src/datautil.py


Functions for Downloading data

In [6]:
%%writefile -a ../src/datautil.py

def get_soup(url):
    
    
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def download_data(url, n=50, sleep=1):
    cpus = [dict() for x in range(n)]
    soup = get_soup(url)
    table = soup.find('div', class_='chart')
    rows = table.find_all('li')
    attributes = ['Class:', 'Socket', 'Clocks', 'Turbo ', 'Cores:', 'Typica']

    i = 0
    for row in rows:
        link = row.find('a').get('href')
        link = 'https://www.cpubenchmark.net/' + link

        soup = get_soup(link)
        price = row.find('span', class_='price-neww').get_text()
        name = row.find('span', class_='prdname').get_text()
        

        cpus[i]["name"] = name
        cpus[i]["price"] = price

        table = soup.find('div', class_= 'left-desc-cpu')
        body = table.find_all('p')
        for col in body:
            text = col.get_text()
            title = col.find('strong').get_text()

            #Verifying attribute is one that we want
            if title[:6] in attributes:
                data = text.split(':')

                #Edge case for cores/threads being in the same exact <p> tag
                if title == "Cores:":

                    #Another edge case for amd cpu's
                    if (text.find("(") != -1):
                        data[1] = data[1][1]
                        data.append("Threads")
                        data.append(data[1])

                    else:
                        temp2 = data[2]
                        data[1] = data[1][1:3]
                        data[2] = "Threads"
                        data.append(temp2[1:])

                    cpus[i][data[2]] = data[3]

                ##Formatting strings
                if title != "Socket:":
                    data[1] = data[1].replace(" ", "")
                else:
                    data[1] = data[1][1:]

                if (title=='Clockspeed:') or (title=='Turbo Speed:'):
                    data[1] = data[1][:-3]

                #Assign to dict
                cpus[i][data[0]] = data[1]
                
        #Gathering scores from other part of dataset
        table = soup.find('div', class_= 'right-desc')
        body = table.find_all('p')
        multi_threaded_score = table.find_all('span')[1].get_text()
        
        data = table.get_text().split(":")
        data[1] = data[1][:5]
        data[1] = data[1].replace(" ", "")
        single_threaded_score = data[1]
        print(name)
        
        cpus[i]["mt_score"] = int(multi_threaded_score)
        cpus[i]["st_score"] = int(single_threaded_score)

        if i >= n-1:
            break
        i = i+1
        time.sleep(sleep)
    
    return pd.DataFrame.from_dict(cpus)

Appending to ../src/datautil.py


Adding local src folder to sys.path

In [3]:
import sys
project_dir = '/home/atoris/course-project-thomas-wright/src'
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)


This next step actaully downloads all the data, this takes a while depnding on how fast you want to reload pages from the website, the rate limit is put into effect to not get blocked from the website.

In [4]:
import os
import pandas as pd
import datautil as du
import importlib

urls = ['https://www.cpubenchmark.net/high_end_cpus.html', 'https://www.cpubenchmark.net/mid_range_cpus.html', 'https://www.cpubenchmark.net/midlow_range_cpus.html', 'https://www.cpubenchmark.net/low_end_cpus.html']

df = pd.DataFrame()
for url in urls:
    df = pd.concat([df,du.download_data(url, 185, .25)])

df.reset_index(inplace=True)

KeyboardInterrupt: 

#### Verify the DataFrame

In [ ]:
df.drop('index', axis=1, inplace=True)
df.info()
df.head(5)


Functions for saving and reloading the dataframe

In [7]:
%%writefile -a ../src/datautil.py

def save_df(df, filename):
    basedir = os.path.dirname(os.getcwd())
    filepath = 'data/' + filename
    full_path = os.path.join(basedir, filepath)
    df.to_csv(full_path, index=False)

def load_data(url, file):
    file = os.path.dirname(os.getcwd()) + '/data/' + file
    if not os.path.exists(file):
        if not os.path.exists(os.path.dirname(file)):
            os.makedirs(os.path.dirname(file))
        df = download_data(url)
        save_df(df, file)
    return pd.read_csv(file)


Appending to ../src/datautil.py


Use these functions to save our data

In [ ]:
importlib.reload(du)

du.save_df(df, 'cpu_data.csv')

Verify new project files have been saved

In [ ]:
!tree ../

Next step is to clean the rest of the data